In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from google.colab import files
 
 


In [ ]:
uploaded = files.upload()

Saving chrome_reviews(1).csv to chrome_reviews(1).csv


In [ ]:
data = pd.read_csv("/content/chrome_reviews(1).csv")

In [ ]:
data.head(5)

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome


In [ ]:
data_2 = data[['ID','Text','Star']]
data_2.head(5)

,ID,Text,Star
0,3886,This is very helpfull aap.,5
1,3887,Good,3
2,3888,Not able to update. Neither able to uninstall.,1
3,3889,Nice app,4
4,3890,Many unwanted ads,1


In [ ]:
data_2['Rate'] = data_2['Star'].apply(lambda x: 1 if x>2 else 0)
data_2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Text,Star,Rate
0,3886,This is very helpfull aap.,5,1
1,3887,Good,3,1
2,3888,Not able to update. Neither able to uninstall.,1,0
3,3889,Nice app,4,1
4,3890,Many unwanted ads,1,0


Preprocessing of the texts and removing stop words

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps=PorterStemmer()


def data_cleaning(data_set):
  corpus=[]
  for i in range(0,len(data_set)):
    
    review=re.sub("[^a-zA-Z]"," ",str(data_set['Text'][i]))

    
    review=review.lower()

    
    review=review.split()

    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]

    
    review=' '.join(review)

    
    corpus.append(review)
      
   
  for i in range(len(corpus)):
    data_set['Text'][i]=corpus[i]

  return data_set

In [ ]:
data_2 = data_cleaning(data_2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
data_2.head(5)

,ID,Text,Star,Rate
0,3886,helpful aap,5,1
1,3887,good,3,1
2,3888,abl updat neither abl uninstal,1,0
3,3889,nice app,4,1
4,3890,mani unwant ad,1,0


In [ ]:
def data_cleaning2():
  global data_2
  
  nan_value = float("NaN")
  data_2.replace("", nan_value, inplace=True)
  data_2=data_2.dropna()

  
  data_2.reset_index(inplace=True)
  corpus=list(data_2['Text'])

In [ ]:
data_cleaning2()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
X=data_2['Text'].values
y=data_2['Rate'].values


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [ ]:
models,model_names,best_scores,best_params,test_score=[],[],[],[],[]

Making Pipeline and selecting the best models

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Creating MultinomialNB model
from sklearn.naive_bayes import MultinomialNB
text_clf_NB = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfNB', MultinomialNB())])

parameters_NB = {'vect__ngram_range': [(1, 1), (1, 2),(1,3)],'tfidf__use_idf': (True, False),'clfNB__alpha': [0.01,0.1,0.15,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]}

MultinomialNB_classifier = GridSearchCV(text_clf_NB, parameters_NB, n_jobs=-1)
MultinomialNB_classifier = MultinomialNB_classifier.fit(X_train,y_train)

NB_model=MultinomialNB_classifier.best_estimator_
y_pred=NB_model.predict(X_test)

model_names.append('MultinomialNB')
models.append(MultinomialNB_classifier)
best_params.append(MultinomialNB_classifier.best_params_)
best_scores.append(MultinomialNB_classifier.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test Accuracy score : ",accuracy_score(y_test,y_pred))
print(MultinomialNB_classifier.best_score_)
print(MultinomialNB_classifier.best_params_)

Test Accuracy score :  0.8461538461538461
0.8566811779686763
{'clfNB__alpha': 0.3, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


In [ ]:
text_clf_Log = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfLog', LogisticRegression())])
parameters_Log = {'vect__ngram_range': [(1, 1), (1, 2),(1,3)],'tfidf__use_idf': (True, False),
                  'clfLog__penalty':['l2','none'],'clfLog__C':[1.0,1.5,2.0],
                  'clfLog__solver':['newton-cg', 'lbfgs', 'sag', 'saga'],
                  'clfLog__multi_class':['ovr']}
LogisticRegression_classifier = GridSearchCV(text_clf_Log, parameters_Log, n_jobs=-1)
LogisticRegression_classifier = LogisticRegression_classifier.fit(X_train,y_train)

Logistic_model=LogisticRegression_classifier.best_estimator_
y_pred=Logistic_model.predict(X_test)

model_names.append('LogisticRegression')
models.append(LogisticRegression_classifier)
best_params.append(LogisticRegression_classifier.best_params_)
best_scores.append(LogisticRegression_classifier.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test accuracy score : ",accuracy_score(y_test,y_pred)) 
print(LogisticRegression_classifier.best_score_)
print(LogisticRegression_classifier.best_params_)

Test accuracy score :  0.8468727534148095
0.8548843943712955
{'clfLog__C': 2.0, 'clfLog__multi_class': 'ovr', 'clfLog__penalty': 'l2', 'clfLog__solver': 'lbfgs', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


In [21]:
from sklearn.linear_model import SGDClassifier
text_clf_SGD = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()),('clfSGD', SGDClassifier())])

parameters_SGD = {'vect__ngram_range': [(1, 1), (1, 2),(1,3),(1,4)],'tfidf__use_idf': (True, False),'clfSGD__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                 'clfSGD__penalty':['l2', 'l1', 'elasticnet'],'clfSGD__alpha':[0.0001,0.001,0.01,0.1,0.2,0.3,0.4,0.5]}

SGD_classifier = GridSearchCV(text_clf_SGD, parameters_SGD, n_jobs=-1)
SGD_classifier = SGD_classifier.fit(X_train,y_train)

SGD_model=SGD_classifier.best_estimator_
y_pred=SGD_model.predict(X_test)

model_names.append('SGDClassifier')
models.append(SGD_classifier)
best_params.append(SGD_classifier.best_params_)
best_scores.append(SGD_classifier.best_score_)
test_score.append(accuracy_score(y_test,y_pred))

print("Test accuracy score : ",accuracy_score(y_test,y_pred))
print(SGD_classifier.best_score_)
print(SGD_classifier.best_params_)

Test accuracy score :  0.8454349388928828
0.855063927294822
{'clfSGD__alpha': 0.001, 'clfSGD__loss': 'modified_huber', 'clfSGD__penalty': 'l2', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}


In [22]:
for i in range(len(test_score)):
  print(model_names[i])
  print(best_scores[i])
  print(best_params[i])
  print('Test accuracy :',test_score[i])
  print('----------------------------------------------------------------------------------------')

MultinomialNB
0.8566811779686763
{'clfNB__alpha': 0.3, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Test accuracy : 0.8461538461538461
----------------------------------------------------------------------------------------
LogisticRegression
0.8548843943712955
{'clfLog__C': 2.0, 'clfLog__multi_class': 'ovr', 'clfLog__penalty': 'l2', 'clfLog__solver': 'lbfgs', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Test accuracy : 0.8468727534148095
----------------------------------------------------------------------------------------
SGDClassifier
0.855063927294822
{'clfSGD__alpha': 0.001, 'clfSGD__loss': 'modified_huber', 'clfSGD__penalty': 'l2', 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Test accuracy : 0.8454349388928828
----------------------------------------------------------------------------------------


Among the 3 models, we tried. We find that Logistic Regression gives the best accuracy 

In [24]:
import joblib
for i in range(len(model_names)):
  if model_names[i]=='LogisticRegression':
    model=models[i]
joblib.dump(model,'Review_savemodel.mod')

['Review_savemodel.mod']